In [109]:
import pandas as pd
import numpy as np
import json

In [172]:
pd.set_option("display.precision", 3)
pd.set_option('future.no_silent_downcasting', True)

In [173]:
df_raw = pd.read_json(
    "../app/public/data/transactions_by_residency_of_buyers_county_level.json",
    orient="columns",
    dtype={"MKOOD": str},
)
mapping = json.load(open("../app/public/data/id_to_county.json"))

In [174]:
updated_rows = []

for i, row in df_raw.iterrows():
    county_id = row["MKOOD"]

    for year, county_data in row.data.items():

        for country_data in county_data:
            updated_rows.append(
                {
                    "county_id": county_id,
                    "county_name": mapping[county_id],
                    "year": year,
                    "country": country_data["Name"],
                    "count": country_data["Number"],
                    "total_area_ha": country_data["Total area (ha)"],
                    "total_value_eur": country_data["Total value (eur)"],
                }
            )

df = pd.DataFrame(updated_rows)

In [175]:
df.total_value_eur = pd.to_numeric(df.total_value_eur.replace('***', np.nan), errors='coerce')
df.total_area_ha = pd.to_numeric(df.total_area_ha.replace('***', np.nan), errors='coerce')

In [176]:
df

,county_id,county_name,year,country,count,total_area_ha,total_value_eur
0,0037,Harju maakond,2010,United States of America,27,26.0,9.429e+05
1,0037,Harju maakond,2010,Estonia,19431,10946.0,9.234e+08
2,0037,Harju maakond,2010,Ireland,9,0.2,3.686e+05
3,0037,Harju maakond,2010,Italy,12,0.1,5.651e+05
4,0037,Harju maakond,2010,Canada,5,0.0,1.696e+05
...,...,...,...,...,...,...,...
1489,0087,Võru maakond,2023,Other residents,12,39.0,1.775e+05
1490,0087,Võru maakond,2023,TOTAL,1850,4881.0,6.035e+07
1491,0087,Võru maakond,2024,Estonia,1982,6120.0,7.677e+07
1492,0087,Võru maakond,2024,Other residents,14,87.0,1.719e+06


In [184]:
df_transformed = df.pivot_table(
    index=['county_name', 'year'],
    columns='country',
    values='total_value_eur',
    aggfunc='sum',
    fill_value=0
)
df_transformed = df_transformed.reset_index()

df_transformed

country,county_name,year,Australia,Austria,Belarus,Belgium,Brazil,Canada,China,Cyprus,...,Sweden,Switzerland,TOTAL,Turkey,Ukraine,United Arab Emirates,United Kingdom,United States of America,Uzbekistan,Virgin Islands (British)
0,Harju maakond,2010,0.0,0.0,279796.0,0.0,0.0,169590.0,0.0,0.0,...,1.942e+06,3.256e+05,9.706e+08,0.0,5.057e+05,0.0,1.162e+06,9.429e+05,0.0,0.0
1,Harju maakond,2011,0.0,0.0,597746.0,731577.0,0.0,0.0,0.0,0.0,...,2.165e+06,4.205e+05,1.189e+09,99824.0,3.113e+05,0.0,6.468e+05,9.824e+05,0.0,0.0
2,Harju maakond,2012,0.0,177000.0,0.0,511200.0,0.0,701424.0,0.0,631800.0,...,1.232e+06,4.502e+05,1.299e+09,0.0,2.020e+05,0.0,1.349e+06,2.042e+05,0.0,0.0
3,Harju maakond,2013,702495.0,0.0,0.0,261745.0,0.0,90999.0,0.0,0.0,...,1.539e+06,1.622e+06,1.542e+09,0.0,3.415e+05,0.0,7.685e+05,5.092e+05,0.0,0.0
4,Harju maakond,2014,0.0,0.0,113900.0,591000.0,0.0,0.0,0.0,0.0,...,1.150e+06,0.000e+00,1.737e+09,0.0,2.172e+06,0.0,2.433e+06,1.364e+06,894000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,Võru maakond,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000e+00,0.000e+00,6.465e+07,0.0,0.000e+00,0.0,0.000e+00,0.000e+00,0.0,0.0
221,Võru maakond,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000e+00,0.000e+00,7.126e+07,0.0,0.000e+00,0.0,0.000e+00,0.000e+00,0.0,0.0
222,Võru maakond,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000e+00,0.000e+00,7.215e+07,0.0,0.000e+00,0.0,0.000e+00,0.000e+00,0.0,0.0
223,Võru maakond,2023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000e+00,0.000e+00,6.035e+07,0.0,0.000e+00,0.0,0.000e+00,0.000e+00,0.0,0.0
